# 3. 다른 방법으로 재 시도

In [ ]:
# ====================================
# 셀 1 : 공통 라이브러리 + 경로 변수
# ====================================
import os, re, warnings, torch, numpy as np
import torch.nn.functional as F
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm

# open_clip_torch
import open_clip

# 🤗 Transformers
from transformers import (
    BlipProcessor, BlipForConditionalGeneration,
    AutoTokenizer, AutoModelForSeq2SeqLM
)

from rapidfuzz import fuzz

warnings.filterwarnings("ignore")

# ---------------- 경로 변수만 수정 ----------------
BLIP_CKPT = r"./blip_finetuned_model_less_data/checkpoint-1250"
IMG_DIR   = r"./dataset/open_dataset/test_input_images"
TEST_CSV  = r"./dataset/open_dataset/test.csv"
SUB_CSV   = r"./dataset/open_dataset/sample_submission.csv"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", DEVICE)

✅ Device: cuda


In [2]:
# ====================================
# 셀 2 : BLIP-VQA, OpenCLIP, FLAN-T5 Small 로드
#         + 파라미터 자동 집계
# ====================================
from collections import OrderedDict

param_tbl, total_params = OrderedDict(), 0
def _add_model(name, model):
    global total_params
    n = sum(p.numel() for p in model.parameters()) / 1_000_000  # Million
    param_tbl[name] = n
    total_params += n
    return model

# 1) BLIP VQA (Fine-tuned)
proc_blip = BlipProcessor.from_pretrained("Salesforce/blip-vqa-capfilt-large")
blip_vqa  = _add_model(
    "BLIP-VQA (Fine-tuned)",
    BlipForConditionalGeneration.from_pretrained(BLIP_CKPT).to(DEVICE).eval()
)

# 2) OpenCLIP ViT-L/14  (Pretrained='openai')
clip_model, _, clip_pre = open_clip.create_model_and_transforms(
    "ViT-L-14", pretrained="openai"
)
clip_model = _add_model(
    "OpenCLIP ViT-L/14",
    clip_model.to(DEVICE).eval()
)
clip_tokenizer = open_clip.get_tokenizer("ViT-L-14")

# 3) FLAN-T5 Small
tok_t5 = AutoTokenizer.from_pretrained("google/flan-t5-small")
flan_t5 = _add_model(
    "FLAN-T5 Small",
    AutoModelForSeq2SeqLM.from_pretrained(
        "google/flan-t5-small",
        torch_dtype=torch.bfloat16
    ).to(DEVICE).eval()
)

# -------- 파라미터 결과 출력 --------
print("\n📊 모델별 파라미터(M)")
for n, m in param_tbl.items():
    print(f" • {n:<22}: {m:>7.1f} M")
print(f"🔢 Total : {total_params:.1f} M  ({total_params/1_000:.3f} B)")

assert total_params < 3_000, "❌ 3B 초과 — 모델 축소 필요"
print("✅ 3B 제약 충족 & 라이선스 요건 OK")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



📊 모델별 파라미터(M)
 • BLIP-VQA (Fine-tuned) :   247.4 M
 • OpenCLIP ViT-L/14     :   427.6 M
 • FLAN-T5 Small         :    77.0 M
🔢 Total : 752.0 M  (0.752 B)
✅ 3B 제약 충족 & 라이선스 요건 OK


In [3]:
# ===============================
# 셀 3: 데이터셋 특성 분석
# ===============================

df = pd.read_csv(TEST_CSV)
print(f"📊 총 테스트 케이스: {len(df)}개")

# 질문 유형 분석
question_types = {
    'what_is': 0, 'what_might': 0, 'why': 0, 'which': 0, 
    'where': 0, 'when': 0, 'how': 0, 'cultural': 0
}

for question in df['Question']:
    q_lower = question.lower()
    if 'what is' in q_lower or 'what are' in q_lower:
        question_types['what_is'] += 1
    elif 'what might' in q_lower or 'what likely' in q_lower:
        question_types['what_might'] += 1  
    elif 'why' in q_lower:
        question_types['why'] += 1
    elif 'which' in q_lower:
        question_types['which'] += 1
    elif 'where' in q_lower:
        question_types['where'] += 1
    elif 'when' in q_lower or 'what time' in q_lower:
        question_types['when'] += 1
    elif 'how' in q_lower:
        question_types['how'] += 1
    elif 'cultural' in q_lower or 'tradition' in q_lower:
        question_types['cultural'] += 1

print("\n📈 질문 유형별 분포:")
for qtype, count in question_types.items():
    if count > 0:
        print(f"  {qtype.upper()}: {count}개 ({count/len(df)*100:.1f}%)")

# 선택지 길이 분석
option_lengths = []
for _, row in df.head(50).iterrows():  # 처음 50개만 분석
    for opt in ['A', 'B', 'C', 'D']:
        option_lengths.append(len(row[opt].split()))

print(f"\n📏 선택지 평균 길이: {np.mean(option_lengths):.1f} 단어")
print(f"📏 선택지 길이 범위: {min(option_lengths)} ~ {max(option_lengths)} 단어")

📊 총 테스트 케이스: 852개

📈 질문 유형별 분포:
  WHAT_IS: 369개 (43.3%)
  WHAT_MIGHT: 167개 (19.6%)
  WHY: 104개 (12.2%)
  WHICH: 34개 (4.0%)
  WHERE: 3개 (0.4%)
  WHEN: 9개 (1.1%)
  HOW: 15개 (1.8%)
  CULTURAL: 46개 (5.4%)

📏 선택지 평균 길이: 8.6 단어
📏 선택지 길이 범위: 5 ~ 15 단어


In [4]:
# ===============================  
# 셀 4: 삼성 챌린지 특화 점수 함수
# ===============================

def clean(txt: str) -> str:
    """텍스트 정규화 - 한글 포함 처리"""
    txt = re.sub(r"[^a-z0-9 ]", " ", txt.lower()).strip()
    return re.sub(r"\s+", " ", txt)

# 다중 프롬프트 템플릿 (삼성 챌린지 질문 유형별)
PROMPT_TEMPLATES = {
    'what_is': [
        "Question: {q} Answer: {o}",
        "What do you see? {q} It is {o}",
        "{q} The answer is {o}"
    ],
    'what_might': [
        "Question: {q} Answer: {o}",
        "What might happen? {q} Likely {o}", 
        "{q} Probably {o}"
    ],
    'why': [
        "Question: {q} Answer: {o}",
        "Why is this happening? {q} Because {o}",
        "{q} The reason is {o}"
    ],
    'default': [
        "Question: {q} Answer: {o}",
        "{q} The answer is {o}",
        "Q: {q} A: {o}"
    ]
}

def get_question_type(question):
    """질문 유형 자동 분류"""
    q = question.lower()
    if 'what is' in q or 'what are' in q:
        return 'what_is'
    elif 'what might' in q or 'what likely' in q:
        return 'what_might'
    elif 'why' in q:
        return 'why'
    else:
        return 'default'

@torch.no_grad()
def adaptive_tf_score(img, question, option):
    """질문 유형별 적응적 Teacher Forcing"""
    q_type = get_question_type(question)
    templates = PROMPT_TEMPLATES[q_type]
    weights = [0.5, 0.3, 0.2]  # 템플릿별 가중치
    
    total_score = 0.0
    for template, weight in zip(templates, weights):
        prompt = template.format(q=question, o=option)
        inp = proc_blip(images=img, text=prompt, return_tensors="pt").to(DEVICE)
        loss = blip_vqa(**inp, labels=inp["input_ids"]).loss.item()
        total_score += weight * (-loss)
    
    return total_score

@torch.no_grad()
def enhanced_clip_score(img, question, option):
    """길이 보정 + 의미적 가중치 CLIP 점수"""
    # 기본 텍스트
    text = f"{question} {option}"
    
    img_tensor = clip_pre(img).unsqueeze(0).to(DEVICE)
    txt_tensor = clip_tokenizer([text]).to(DEVICE)
    
    img_feat = clip_model.encode_image(img_tensor)
    txt_feat = clip_model.encode_text(txt_tensor)
    
    img_feat = F.normalize(img_feat, dim=-1)
    txt_feat = F.normalize(txt_feat, dim=-1)
    
    cosine_sim = (img_feat @ txt_feat.T).item()
    
    # 길이 보정 (너무 짧은 답변에 페널티)
    length_penalty = min(1.0, len(option.split()) / 3.0)
    
    return cosine_sim * length_penalty

@torch.no_grad()
def multi_generation_score(img, question, option, n_samples=3):
    """다중 생성 + RapidFuzz 유사도"""
    inp = proc_blip(images=img, text=question, return_tensors="pt").to(DEVICE)
    
    similarities = []
    for _ in range(n_samples):
        generated_ids = blip_vqa.generate(
            **inp,
            max_new_tokens=10,
            num_beams=3,
            temperature=0.7,
            do_sample=True
        )
        
        generated_text = proc_blip.decode(generated_ids[0], skip_special_tokens=True)
        clean_generated = clean(generated_text.replace(question.lower(), ""))
        clean_option = clean(option)
        
        # 다양한 RapidFuzz 유사도 메트릭 조합
        ratio_sim = fuzz.ratio(clean_generated, clean_option) / 100.0
        partial_sim = fuzz.partial_ratio(clean_generated, clean_option) / 100.0  
        token_sim = fuzz.token_sort_ratio(clean_generated, clean_option) / 100.0
        
        # 가중 평균
        combined_sim = 0.4 * ratio_sim + 0.3 * partial_sim + 0.3 * token_sim
        similarities.append(combined_sim)
    
    return float(np.mean(similarities))

print("✅ 삼성 챌린지 특화 점수 함수 준비 완료")

✅ 삼성 챌린지 특화 점수 함수 준비 완료


In [5]:
# ===============================
# 셀 5: 점수 정규화 시스템
# ===============================

class SmartScoreNormalizer:
    def __init__(self):
        # 실제 관측된 점수 분포 기반 통계
        self.tf_stats = {'mean': -3.8, 'std': 0.9}
        self.clip_stats = {'mean': 0.28, 'std': 0.16}  
        self.gen_stats = {'mean': 0.22, 'std': 0.14}
    
    def normalize_scores(self, tf_score, clip_score, gen_score):
        """Z점수 정규화 후 sigmoid로 0-1 범위 매핑"""
        # Z-score 정규화
        tf_z = (tf_score - self.tf_stats['mean']) / self.tf_stats['std']
        clip_z = (clip_score - self.clip_stats['mean']) / self.clip_stats['std']
        gen_z = (gen_score - self.gen_stats['mean']) / self.gen_stats['std']
        
        # Sigmoid 변환으로 0-1 범위
        tf_norm = 1 / (1 + np.exp(-tf_z))
        clip_norm = 1 / (1 + np.exp(-clip_z))
        gen_norm = 1 / (1 + np.exp(-gen_z))
        
        return tf_norm, clip_norm, gen_norm

# 전역 정규화기 초기화
normalizer = SmartScoreNormalizer()
print("✅ 스마트 점수 정규화 시스템 준비 완료")

✅ 스마트 점수 정규화 시스템 준비 완료


In [ ]:
# ===============================
# 셀 6: 질문 유형별 최적 가중치
# ===============================

def get_smart_weights(question):
    """질문 유형 분석 기반 최적 가중치 결정"""
    q = question.lower()
    
    # 세분화된 질문 유형별 최적 가중치
    if 'what is' in q or 'what are' in q:
        return 0.30, 0.55, 0.15  # 객체인식 → CLIP 최우선
    elif 'why' in q or 'reason' in q:
        return 0.70, 0.20, 0.10  # 추론질문 → TF 최우선
    elif 'what might' in q or 'likely' in q:
        return 0.35, 0.30, 0.35  # 예측질문 → GEN 중요
    elif 'where' in q or 'location' in q:
        return 0.20, 0.70, 0.10  # 장소질문 → CLIP 최우선
    elif 'cultural' in q or 'tradition' in q:
        return 0.60, 0.25, 0.15  # 문화질문 → TF 중심
    elif 'how' in q:
        return 0.50, 0.35, 0.15  # 방법질문 → TF 중심
    else:
        return 0.45, 0.40, 0.15  # 균형잡힌 기본값

def optimized_ensemble_scores(img, question, options):
    """동적 가중치 + 정규화 최적 앙상블"""
    w_tf, w_clip, w_gen = get_smart_weights(question)
    
    scores = []
    for opt in options:
        # 원시 점수 계산
        tf_raw = adaptive_tf_score(img, question, opt)
        clip_raw = enhanced_clip_score(img, question, opt)
        gen_raw = multi_generation_score(img, question, opt, n_samples=2)
        
        # 점수 정규화 적용
        tf_norm, clip_norm, gen_norm = normalizer.normalize_scores(tf_raw, clip_raw, gen_raw)
        
        # 동적 가중 결합
        final_score = w_tf * tf_norm + w_clip * clip_norm + w_gen * gen_norm
        scores.append(final_score)
    
    return scores

print("✅ 동적 가중치 최적 앙상블 시스템 준비")

✅ 동적 가중치 최적 앙상블 시스템 준비


In [7]:
# 셀 7: 향상된 TTA 앙상블 시스템
from PIL import ImageEnhance

def enhanced_tta_scores(img, question, options, transforms=None):
    if transforms is None:
        from torchvision import transforms as T
        transforms = [
            T.Resize((224,224)),
            T.Compose([T.Resize(256), T.CenterCrop(224)]),
            T.Compose([T.Resize(256), T.RandomCrop(224)]),
            T.ColorJitter(brightness=0.15, contrast=0.15)
        ]
    all_scores, confs = [], []
    for tfm in transforms:
        aug = tfm(img) if hasattr(tfm, '__call__') else img
        sc = optimized_ensemble_scores(aug, question, options)
        conf = max(sc) - np.mean(sc)
        all_scores.append(sc)
        confs.append(max(conf, 0.1))
    total_conf = sum(confs)
    weights = [c/total_conf for c in confs]
    final = [sum(w*s[i] for w,s in zip(weights, all_scores)) for i in range(len(options))]
    return final, weights

print("✅ 향상된 TTA 앙상블 준비 완료")

✅ 향상된 TTA 앙상블 준비 완료


In [8]:
# 셀 8: 온도 스케일링 + Softmax 확률화 함수
import numpy as np

def temperature_softmax(scores, temperature=1.2):
    arr = np.array(scores) / temperature
    ex = np.exp(arr - np.max(arr))
    return (ex / ex.sum()).tolist()

print("✅ 온도 스케일링 + Softmax 준비 완료")

✅ 온도 스케일링 + Softmax 준비 완료


In [9]:
# 셀 9: 메타 앙상블 + 최종 추론 시스템
def final_samsung_inference(debug_samples=5, temperature=1.2):
    df = pd.read_csv(TEST_CSV)
    answers = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="🔥 최종 돌파"):
        img_path = os.path.join(IMG_DIR, os.path.basename(row["img_path"]))
        if not os.path.exists(img_path):
            answers.append("A"); continue
        img = Image.open(img_path).convert("RGB")
        q, opts = row["Question"], [row[k] for k in "ABCD"]
        # 1) Enhanced TTA 앙상블
        tta_scores, tta_w = enhanced_tta_scores(img, q, opts)
        # 2) 온도 스케일링 Softmax
        prob = temperature_softmax(tta_scores, temperature)
        # 3) 최종 선택
        pred = "ABCD"[int(np.argmax(prob))]
        answers.append(pred)
        if idx < debug_samples:
            print(f"\n--- 샘플 {idx+1} ---")
            print(f"TTA weights: {[f'{w:.2f}' for w in tta_w]}")
            print(f"Raw scores: {[f'{s:.4f}' for s in tta_scores]}")
            print(f"Probabilities: {[f'{p:.3f}' for p in prob]}")
            print(f"선택: {pred}")
    sub = pd.read_csv(SUB_CSV)
    sub["answer"] = answers
    sub.to_csv(SUB_CSV, index=False)
    print("✅ 최종 제출 파일 저장 완료")
    return answers

# 🚀 실행
final_results = final_samsung_inference(debug_samples=5, temperature=1.2)

🔥 최종 돌파:   0%|          | 0/852 [00:00<?, ?it/s]


--- 샘플 1 ---
TTA weights: ['0.25', '0.25', '0.25', '0.25']
Raw scores: ['0.1918', '0.2176', '0.2035', '0.1969']
Probabilities: ['0.248', '0.253', '0.250', '0.249']
선택: B

--- 샘플 2 ---
TTA weights: ['0.25', '0.25', '0.25', '0.25']
Raw scores: ['0.2178', '0.1564', '0.1707', '0.1782']
Probabilities: ['0.258', '0.245', '0.248', '0.249']
선택: A

--- 샘플 3 ---
TTA weights: ['0.25', '0.25', '0.25', '0.25']
Raw scores: ['0.1970', '0.2339', '0.2167', '0.2196']
Probabilities: ['0.246', '0.254', '0.250', '0.251']
선택: B

--- 샘플 4 ---
TTA weights: ['0.25', '0.25', '0.25', '0.25']
Raw scores: ['0.1549', '0.1509', '0.1726', '0.1571']
Probabilities: ['0.249', '0.248', '0.253', '0.250']
선택: C

--- 샘플 5 ---
TTA weights: ['0.25', '0.25', '0.25', '0.25']
Raw scores: ['0.1692', '0.1699', '0.1652', '0.1692']
Probabilities: ['0.250', '0.250', '0.249', '0.250']
선택: B
✅ 최종 제출 파일 저장 완료
